## Summarizing parking tickets by year and infraction type

A few simple plots and outputting the summary table to .csv

In [1]:
import pandas as pd
import numpy as np
import os
import zipfile
import altair as alt

In [34]:
years = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
dfid_all = []
for year in years:
    
    # get tickets for the year
    dfs = []
    zf = zipfile.ZipFile("parking-tickets/parking-tickets-" + str(year) + ".zip")
    csvs = zf.namelist()
    for csv in csvs:
        dfs.append(pd.read_csv(zf.open(csv), engine="python", on_bad_lines='skip'))
        print(csv)
    df = pd.concat(dfs)
    del dfs
    del df["tag_number_masked"], df["province"]
    df["year"] = year
    
    # compute counts by type
    dfid = pd.DataFrame(df.value_counts(["infraction_description"]))
    dfid.columns=["count"]
    dfid = dfid.rename_axis("infraction_description").reset_index()
    
    # sum value of tickets
    dff = df.groupby(['infraction_description'])['set_fine_amount'].sum().reset_index()
    
    # merge 
    dfid = pd.merge(left = dfid, right = dff, on = 'infraction_description')
        
    dfid["year"] = year
  
    dfid_all.append(dfid)

dfid = pd.concat(dfid_all)
dfid.to_csv("summary-year-type.csv")

Parking_Tags_data_2011.csv
Parking_Tags_Data_2012.csv
Parking_Tags_Data_2013.csv
Parking_Tags_Data_2014_2.csv
Parking_Tags_Data_2014_3.csv
Parking_Tags_Data_2014_4.csv
Parking_Tags_Data_2014_1.csv
Parking_Tags_Data_2015_1.csv
Parking_Tags_Data_2015_2.csv
Parking_Tags_Data_2015_3.csv
Parking_Tags_Data_2016_2.csv
Parking_Tags_Data_2016_3.csv
Parking_Tags_Data_2016_4.csv
Parking_Tags_Data_2016_1.csv
Parking_Tags_Data_2017_1.csv
Parking_Tags_Data_2017_2.csv
Parking_Tags_Data_2017_3.csv
Parking_Tags_Data_2018_1.csv
Parking_Tags_Data_2018_2.csv
Parking_Tags_Data_2018_3.csv
Parking_Tags_Data_2019.000.csv
Parking_Tags_Data_2019.001.csv
Parking_Tags_Data_2019.002.csv
Parking_Tags_Data_2019.003.csv
Parking_Tags_Data_2019.004.csv
Parking_Tags_Data_2019.005.csv
Parking_Tags_Data_2019.006.csv
Parking_Tags_Data_2019.007.csv
Parking_Tags_Data_2019.008.csv
Parking_Tags_Data_2020.000.csv
Parking_Tags_Data_2020.001.csv
Parking_Tags_Data_2020.002.csv
Parking_Tags_Data_2020.003.csv
Parking_Tags_Data_2020.

In [5]:
dfid = pd.read_csv("summary-year-type.csv")

In [6]:
dfid.groupby(['infraction_description'])['count'].sum().reset_index().sort_values(by='count').to_csv("infraction_types.csv")

In [7]:
dfy = dfid.groupby(["year"]).sum()
dfy = dfy[["count", "set_fine_amount"]]
dfy.reset_index(inplace=True)
dfy = dfy.rename(columns = {'index':'year'})
dfy.to_json("tickets-by-year.json",orient='records')

### Total number of tickets by year

In [16]:
dfy = dfid.groupby(['year'])['count'].sum().reset_index()
dfy["date"] = dfy["year"].astype(str) + "-01-01"
dfy

alt.Chart(dfy).mark_line(
     point=alt.OverlayMarkDef()
).encode(
    x=alt.X('date:T', title="Year", axis=alt.Axis(
        labelAngle=45,
        gridOpacity=0.5
    ), scale=alt.Scale(domain=["2010-06-01", "2022-06-01"])),
    y=alt.Y('count', title="Number Of Parking Tickets")
)

alt.Chart(...)

### Total fines ($) per year

In [15]:
dfy = dfid.groupby(['year'])['set_fine_amount'].sum().reset_index()
dfy["date"] = dfy["year"].astype(str) + "-01-01"
dfy

alt.Chart(dfy).mark_line(
     point=alt.OverlayMarkDef()
).encode(
    x=alt.X('date:T', title="Year", axis=alt.Axis(
        labelAngle=45,
        gridOpacity=0.5
    ), scale=alt.Scale(domain=["2010-06-01", "2022-06-01"])),
    y=alt.Y('set_fine_amount', title="Fines ($) per Year")
)

alt.Chart(...)

In [10]:
# total income
dfid["set_fine_amount"].sum()

1214153740

In [12]:
dfid

,Unnamed: 0,infraction_description,count,set_fine_amount,year
0,0,PARK PROHIBITED TIME NO PERMIT,452882,13586460,2011
1,1,PARK HWY PROHIBED TIME/DAY,380297,15211880,2011
2,2,PARK FAIL TO DISPLAY RECEIPT,346126,10383780,2011
3,3,PARK/LEAVE ON PRIVATE PROPERTY,336661,10099830,2011
4,4,PARK FAIL TO DEP. FEE MACHINE,234060,7021800,2011
...,...,...,...,...,...
2747,158,PARK ON PRIV PROP NO CONSENT,1,0,2022
2748,159,PARK PRIVATE PROP NO CONSENT,1,0,2022
2749,160,PARK PRIVATE VEHICLE ON SITE,1,500,2022
2750,161,PARK MOTORCYCLE - IN SPACE,1,0,2022


### Total number of infractions by year for a specific type

In [37]:
# STAND SIGNED TRANSIT STOP
# STOP VEH OTR THN BCYCL-BYCL LN|STOP NON-BICYCLE IN CYCLE TRCK|PARK PRO VEH ON BICYCLE PATH

dfl = dfid[dfid["infraction_description"].str.contains(
    "STOP VEH OTR THN BCYCL-BYCL LN|STOP NON-BICYCLE IN CYCLE TRCK|PARK PRO VEH ON BICYCLE PATH")].groupby(
    ['year'])['count'].sum().reset_index()
dfl["date"] = dfl["year"].astype(str) + "-01-01"

alt.Chart(dfl, title="# Of Tickets For Parking In Bike Lanes/Paths/Tracks", width=500).mark_line(
     point=alt.OverlayMarkDef()
).encode(
    x=alt.X('date:T', title="Year", axis=alt.Axis(
        labelAngle=45,
        gridOpacity=0.5
    ), scale=alt.Scale(domain=["2010-06-01", "2022-06-01"])),
    y=alt.Y('count', title="")
)

alt.Chart(...)

### Percent of loading infractions by year

In [38]:
dfb = pd.merge(dfy,dfl,on="year")
dfb["load_perc"] = dfb.count_y / dfb.count_x
dfb

alt.Chart(dfb, title="% Of Tickets For Parking In Bike Lanes/Paths/Tracks", width=500).mark_line(
     point=alt.OverlayMarkDef()
).encode(
    x=alt.X('date_x:T', title="Year", axis=alt.Axis(
        labelAngle=45,
        gridOpacity=0.5
    ), scale=alt.Scale(domain=["2010-06-01", "2022-06-01"])),
    y=alt.Y('load_perc', title="", axis=alt.Axis(format='%'))
)

alt.Chart(...)